# Analysis of Plant Growth Experiment
_______________________________________
This notebook analyzes the results of an experiment on plant growth, where the goal is to compare the dried weight of plants under three conditions:

Control group (`ctrl`)<br>
Treatment 1 (`trt1`)<br>
Treatment 2 (`trt2`)<br>

The dataset, PlantGrowth, consists of thirty observations with two variables:<br>
**weight**: Numeric values representing the dried weight of plants.<br>
**group**: A factor variable with three levels (ctrl, trt1, and trt2), indicating the treatment applied.<br>

Statistical tests will be performed, including:
**$t$-tests**: To compare the mean weights between specific groups.
**ANOVA**: To assess whether significant differences in mean weights exist across all three groups.
The analysis is aimed at determining whether the treatment conditions have a significant effect on plant growth compared to the control group.
____________________________

In [32]:
# Importing standard libraries

import math  # Mathematical functions
import random  # Functions for generating random numbers
import pandas as pd  # Library for data manipulation and analysis

# Working with numerical arrays
import numpy as np

# Libraries for data visualization
import matplotlib.pyplot as plt  # Basic plotting
import seaborn as sns  # Advanced statistical plotting

# Libraries for statistical analysis
import itertools  # Functions for creating iterator-based operations
from scipy import stats  # Statistical functions
from scipy.stats import shapiro  # Shapiro-Wilk test for normality

# One-way ANOVA function
from scipy.stats import f_oneway

## Data description

In [33]:
## Saving the data
data = pd.read_csv("PlantGrowth.csv")
#data

The data is stored in a format where the groups are not split into different columns. 

In [34]:
# Splitting into three groups.

# The Control group's data
control = data[data['group']=='ctrl']['weight']

# The Treatment one data
trit1 = control = data[data['group']=='trt1']['weight']

# The Treatment two data
trit2 = control = data[data['group']=='trt2']['weight']


In [ ]:
# The describe() method provides a statistical summary of the data
# pd.concat concatenates data for better visualization
describe_results = pd.concat([control.describe(),trit1.describe(), trit2. describe()], axis=1)
describe_results.columns= ['control', 'trit1','trit2']
describe_results

,control,trit1,trit2
count,10.000000,10.000000,10.000000
mean,5.526000,4.661000,5.526000
std,0.442573,0.793676,0.442573
min,4.920000,3.590000,4.920000
25%,5.267500,4.207500,5.267500
50%,5.435000,4.550000,5.435000
75%,5.735000,4.870000,5.735000
max,6.310000,6.030000,6.310000


The `trit1` group has a lower average weight (4.66) compared to the `control` and `trit2` groups (both around 5.53).<br>
The `trit1` group also shows more variability in weight (higher standard deviation of 0.79), meaning the weights are more spread out, whereas the other two groups have lower variability (around 0.44).<br>
The `trit1` group also has a wider range of weights, starting from 3.59 (the lowest value) to 6.03 (the highest), while the control and trit2 groups have a more consistent range between 4.92 and 6.31.
In conclusion, the trit1 group stands out with lower and more variable weights, which could indicate that the treatment or condition applied to this group had a different effect compared to the other two groups.

## $t$-test for Independent Groups

The $t$-test is used to determine if the difference between the means of two groups is statistically significant.